In [ ]:
!pip install --upgrade diffusers

In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.6 MB/s eta 0:00:00


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.1 MB/s eta 0:00:00


In [ ]:
import torch
from PIL import Image
from io import BytesIO
import requests
from diffusers import StableDiffusionDepth2ImgPipeline
from matplotlib import pyplot as plt
import pandas as pd
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import os
os.chdir("/content/gdrive/My Drive/ComputerVision/")

In [ ]:
# This part is when you are using GPU of Nvidia.
device = torch.device("cpu")
# If a GPU is available, set the device to GPU.
if torch.cuda.is_available:
  device = torch.device("cuda")
  print("GPU is available")
else:
  print("GPU is not available, CPU used")

GPU is available


In [ ]:
# Creating a variable instance of the pipeline
pipe = StableDiffusionDepth2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-depth",
    torch_dtype=torch.float16,
)
# Assigning to GPU
pipe.to(device)

In [ ]:
config = pd.read_csv('config.csv')

In [ ]:
def plot_multiple_images(row, nums_in_row, images, figsize=(5, 4)):
    # Create a figure and subplots with a row of images
    fig, axs = plt.subplots(row, nums_in_row, figsize=figsize)
    # Adjust spacing between subplots
    fig.subplots_adjust(hspace=0, wspace=0)
    if axs.ndim == 1: axs = axs.reshape((1,-1))
    capabilities = row * nums_in_row
    min_one = min(capabilities, len(images))
    for i in range(min_one):
        image = images[i]
        row = i // nums_in_row
        column = i % nums_in_row
        axs[row][column].imshow(image)
        axs[row][column].axis('off')

    # Adjust the spacing between subplots
    # plt.tight_layout()
    plt.show()
    return fig, axs


In [ ]:
originals = []
synthetics = []
for index, row in config.iterrows():
  if index % 70 != 0: continue
  # Read the image path from the current row
  image_path = row['path']
  # Load the image using OpenCV

  image = Image.open(image_path)
  image.resize((256, 256))
  originals.append(image)
  prompt = "blue eyes"
  n_prompt = ''
  images = pipe(prompt=prompt,negative_prompt=n_prompt, image=image, strength=1.0, guidance_scale=1).images
  synthetics.append(images[0])

  if index >= 350: break

s = plot_multiple_images(2, 6, originals + synthetics)

# images[0].save("fantasy_landscape.png")


In [65]:
for m in range(20):
  s = []
  for i in synthetics:
    images = pipe(prompt=prompt,negative_prompt=n_prompt, image=i, strength=0.1, guidance_scale=1).images
    s.append(images[0])
  x = plot_multiple_images(2, 6, originals + s)
  synthetics = s

Output hidden; open in https://colab.research.google.com to view.